# Imaging overviews in spiral

In [ ]:
import specpy

from autosted import AcquisitionPipeline
from autosted.callback_buildingblocks import (
    FOVSettingsGenerator,
    JSONSettingsLoader,
    LocationRemover,
    NewestDataSelector,
    SimpleManualOffset,
    SpiralOffsetGenerator,
    StageOffsetsSettingsGenerator,
)
from autosted.detection import SimpleFocusPlaneDetector
from autosted.imspector import get_current_stage_coords
from autosted.stoppingcriteria import MaximumAcquisitionsStoppingCriterion
from autosted.taskgeneration import AcquisitionTaskGenerator

In [ ]:
# where to save & whether to save combined HDF5 file
save_folder = "acquisition_data/spiral-test"
save_hdf5 = True

# path of measurement parameters (dumped to JSON file)
measurement_parameters = "examples/config_json/20241010_overview_3d_640.json"
# Alternative: get current parameters
# measurement_parameters = (
#     specpy.get_application().value_at("", specpy.ValueTree.Measurement).get()
# )

# yx move size between images in spiral
move_size = [50e-6, 50e-6]

# channel to focus in
focus_channel = 0

# manual offset (zyx) to focus
manual_focus_offset = [0, 0, 0]

In [ ]:
# get current coordinates and print, so we can go back to that position
start_coords = get_current_stage_coords()
print(start_coords)

Now, we instantiate a pipeline an an ```AcquisitionTaskGenerator``` to generate the next position to image at.

In addition to the example in ```basics.ipynb```, we add two optional building blocks:

1. a ```FOVSettingsGenerator``` that sets image size and/or pixel size to user-defined values (e.g. size of movement in spiral)
2. a ```SimpleFocusPlaneDetector``` that returns updated z focus positions using a simple intensity-based autofocus. It can additionally be wrapped in a ```SimpleManualOffset``` block to add a manual offset to the detected highest intensity plane.

In [ ]:
# build pipeline object (1 level: 'field')
pipeline = AcquisitionPipeline(
    save_folder, ["field"], save_combined_hdf5=save_hdf5
)

# callback that will create an acquisition task with given measurement parameters
# at the next stage coordinates in the coordinate list (the next 'position')
next_position_generator = AcquisitionTaskGenerator(
    "field",
    # 1. load basic measurement parameters from file
    LocationRemover(JSONSettingsLoader(measurement_parameters)),
    # 2. (optional) update FOV to match spiral move size in yx (leave z & pixel size as-is -> None)
    FOVSettingsGenerator(lengths=[None] + move_size, pixel_sizes=None),
    # 3. get next position, wrap values as stage offset in parameter dict
    # NOTE: old version, by now, the inner SpiralOffsetGenerator will return a parameter dict by default
    # so this will do the same: SpiralOffsetGenerator(move_size, start_coords[1:])
    StageOffsetsSettingsGenerator(
        SpiralOffsetGenerator(move_size, start_coords[1:], return_parameter_dict=False)
    ),
    # 4. (optional) adjust focus based on last image
    # nested callbacks are best read inside-out:
    # get latest data, get focus plane, optionally add manual offset, wrap values in offset settings dict
    StageOffsetsSettingsGenerator(
        SimpleManualOffset(
            SimpleFocusPlaneDetector(
                NewestDataSelector(pipeline, level="field"), channel=focus_channel
            ),
            offset=manual_focus_offset,
        )
    ),
)

# attach callback so that after each position, the next one will be enqueued
pipeline.add_callback(next_position_generator, "field")

# set maximum number of acquisitions before stop
pipeline.add_stopping_condition(MaximumAcquisitionsStoppingCriterion(50))

# start with initial task from callback
pipeline.run(next_position_generator)